<a href="https://colab.research.google.com/github/AsifAlFaisal/sen2-gee/blob/main/Sen2_SR_MSI_Image_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

# Viz Func

In [5]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

def visualize_image(col):
    # Mosaic the image collection.
    #img = col.mosaic()

    # Create a folium map object.
    center = aoi.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12, height=500, width=500)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

# AOI

In [6]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              92.097050239336184,
              21.241923195974895
            ],
            [
              92.229083686509014,
              21.243567581915894
            ],
            [
              92.22968129424477,
              21.125398839669575
            ],
            [
              92.097400993293448,
              21.123881689028394
            ],
            [
              92.097050239336184,
              21.241923195974895
            ]
          ]
        ]
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# S2 SR/MSI Collection Build

In [7]:
def get_S2_col(aoi, start_date, end_date, cloud_filter, collection="S2_SR"):
  """
  collection -  S2_SR or S2_MSI
  """
  if collection=='S2_SR':
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterBounds(aoi)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))
    return s2_sr_col
  else:  
    s2_msi_col = (ee.ImageCollection('COPERNICUS/S2')\
        .filterBounds(aoi)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))
    return s2_msi_col

# S2 SR Download

In [52]:
ST_DT = '2019-08-20'
ED_DT = '2019-09-20'
CLOUD_FILTER = 60


s2_sr = get_S2_col(aoi, ST_DT, ED_DT, CLOUD_FILTER, collection="S2_SR")
s2_sr_count = s2_sr.size().getInfo()
print(f"Sentinel-2 SR- number of images within given dates: {s2_sr_count}")
import requests
for i in range(s2_sr_count):
  img = ee.Image(s2_sr.toList(1,i).get(0)).clip(aoi)
  #visualize_image(img)
  bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
  link = img.select(bands).getDownloadURL({
          'scale':20,
          'crs':'EPSG:32646',
          'fileFormat': 'GeoTIFF',
          'filePerBand':False,
          'region':aoi
          })
  dl = requests.get(link, allow_redirects=True)
  filename = f"S2SR_{img.getInfo()['properties']['system:index'][:8]}"
  open(f"{filename}.zip", 'wb').write(dl.content)
  #visualize_image(img)

Sentinel-2 SR- number of images within given dates: 2


# S2 MSI Download

In [11]:
ST_DT = '2019-08-20'
ED_DT = '2019-09-20'
CLOUD_FILTER = 60


s2_msi = get_S2_col(aoi, ST_DT, ED_DT, CLOUD_FILTER, collection="S2_MSI")
s2_msi_count = s2_msi.size().getInfo()
print(f"Sentinel-2 MSI- number of images within given dates: {s2_msi_count}")

import requests
for i in range(s2_msi_count):
  img = ee.Image(s2_msi.toList(1,i).get(0)).clip(aoi)
  #visualize_image(img)
  bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
  link = img.select(bands).getDownloadURL({
          'scale':20,
          'crs':'EPSG:32646',
          'fileFormat': 'GeoTIFF',
          'filePerBand':False,
          'region':aoi
          })
  dl = requests.get(link, allow_redirects=True)
  filename = f"S2MSI_{img.getInfo()['properties']['system:index'][:8]}"
  open(f"{filename}.zip", 'wb').write(dl.content)
  #visualize_image(img)

Sentinel-2 MSI- number of images within given dates: 3
